<h1> Preprocessing Data </h1>

<h2> Import necessary libraries </h2>

In [1]:
import numpy as np
import pandas as pd
import re
from datetime import datetime as dt

<h2>Read raw data from file <font color="blue">"steam_game.csv" </font></h2>


In [2]:
df = pd.read_csv('steam_game.csv')
df.head(10)

,Title,Genre,Tags,withDLC,isMature,Franchise,ReleaseDate,Developer,Publisher,Languages,PositiveReviews,TotalReviews,NegativeReviews,OriginalPrice,DiscountPercent
0,Endless Monday: Dreams and Deadlines,"['Adventure', 'Indie']","['Visual Novel', 'Choose Your Own Adventure', ...",False,False,Endless Monday,"May 4, 2023",hcnone,hcnone,"['English', 'Japanese']",722,722,0.0,"142.000,00₫",-20%
1,Senren＊Banka,"['Adventure', 'Casual']","['Visual Novel', 'Casual', 'NSFW', 'Dating Sim...",False,True,NaN,"Feb 14, 2020",YUZUSOFT,HIKARI FIELD,"['English', 'Japanese', 'Simplified Chinese', ...","(16,973)","(17,121)",148.0,"280.000,00₫",NaN
2,A Short Hike,"['Adventure', 'Indie']","['Exploration', 'Adventure', 'Cute', 'Relaxing...",False,False,NaN,"30 Jul, 2019",adamgryu,adamgryu,"['English', 'French', 'Spanish - Latin America...","(14,676)","(14,786)",110.0,"100.000,00₫",NaN
3,Aseprite,"['Animation & Modeling', 'Design & Illustratio...","['Pixel Graphics', 'Design & Illustration', 'A...",False,False,NaN,"Feb 22, 2016",David Capello,Igara Studio,['English'],"(15,469)","(15,597)",128.0,"188.000,00₫",NaN
4,Cats Hidden in Paris,"['Casual', 'Indie', 'Free to Play']","['Cats', 'Hidden Object', 'Puzzle', 'Wholesome...",True,False,Travellin Cats,"16 Jul, 2023",Travellin Cats,Travellin Cats,"['English', 'French', 'Italian', 'German', 'Sp...","(11,336)","(11,435)",99.0,Free,NaN
5,Papa's Freezeria Deluxe,"['Action', 'Casual', 'Indie', 'Simulation', 'S...","['Cooking', 'Time Management', 'Simulation', '...",False,False,Papa Louie,"Mar 31, 2023",Flipline Studios,Flipline Studios,['English'],"(6,441)","(6,487)",46.0,"87.500,00₫",NaN
6,TOEM,"['Adventure', 'Casual', 'Indie']","['Cozy', 'Wholesome', 'Adventure', 'Puzzle', '...",True,False,NaN,"17 Sep, 2021",Something We Made,Something We Made,"['English', 'French', 'German', 'Spanish - Spa...","(4,589)","(4,625)",36.0,"260.000,00₫",NaN
7,Ib,"['Adventure', 'Indie']","['Horror', '2D', 'RPGMaker', 'Psychological Ho...",False,False,PLAYISM,"Apr 11, 2022",kouri,PLAYISM,"['English', 'Japanese', 'Simplified Chinese', ...","(3,048)","(3,077)",29.0,"150.000,00₫",NaN
8,Lil Gator Game,"['Action', 'Adventure', 'Casual', 'Indie']","['Casual', 'Cozy', 'Relaxing', 'Cute', 'Wholes...",False,False,Playtonic Friends,"14 Dec, 2022",MegaWobble,Playtonic Friends,"['English', 'French', 'German', 'Portuguese - ...","(3,338)","(3,369)",31.0,"260.000,00₫",NaN
9,An Arcade Full of Cats,"['Casual', 'Indie', 'Free to Play']","['Cats', 'Hidden Object', 'Relaxing', 'Wholeso...",True,False,Devcats,"16 Nov, 2023",Devcats,Devcats,"['English', 'French', 'Italian', 'German', 'Sp...","(2,711)","(2,736)",25.0,Free,NaN


<h2>Ta xem xét có tổng cộng bao nhiêu hàng và cột thuộc tính ở tập dữ liệu thô </h2>
Nếu dữ liệu có trên 1000 dòng và phải có trên 5 cột thuộc tính thì dữ liệu mới đủ để tiến hành các bước khai thác, xây dựng mô hình học máy, học sâu, ... 

Ngược lại, dữ liệu quá ít, không đủ -> cần thu thập thêm dữ liệu

In [3]:
num_rows, num_cols = df.shape
print(f'Current shape of data: {df.shape}')
if (num_rows > 100) & (num_cols > 5):
    print('Data is good !')
else:
    print('You should collect more data')

Current shape of data: (2021, 15)
Data is good !


### Ta xem xét ý nghĩa của từng cột dữ liệu
1. Title: Tên trò chơi điện tử trên website steam
2. Genre: Thể loại trò chơi
3. Tags: các nhãn trò chơi mà người dùng(người chơi) gắn cho trò chơi
4. withDLC: trò chơi có kèm theo các downloadable content hay không ?	
5. isMature: trò chơi có được gán nhãn không phù hợp cho trẻ nhỏ hay không?
6. Franchise: tên thương hiệu của trò chơi	
7. ReleaseDate: Ngày phát hành của trò chơi	
8. Developer: Nhà phát triển trò chơi	
9. Publisher: Nhà phát hành trò chơi
10. Languages: Các ngôn ngữ được hỗ trợ trong trò chơi	
11. PositiveReviews: Số đánh giá tích cực về trò chơi	
12. TotalReviews: Tổng số đánh giá của người chơi về trò chơi
13. NegativeReviews: Số đánh giá tiêu cực về trò chơi
14. OriginalPrice: Giá bán gốc của trò chơi
15. DiscountPercent: Giảm giá áp dụng trên giá gốc(%)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2021 entries, 0 to 2020
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Title            1734 non-null   object
 1   Genre            1734 non-null   object
 2   Tags             1734 non-null   object
 3   withDLC          1734 non-null   object
 4   isMature         1734 non-null   object
 5   Franchise        883 non-null    object
 6   ReleaseDate      1733 non-null   object
 7   Developer        1734 non-null   object
 8   Publisher        1728 non-null   object
 9   Languages        1734 non-null   object
 10  PositiveReviews  1734 non-null   object
 11  TotalReviews     1734 non-null   object
 12  NegativeReviews  1734 non-null   object
 13  OriginalPrice    2021 non-null   object
 14  DiscountPercent  168 non-null    object
dtypes: object(15)
memory usage: 237.0+ KB


##### Loại bỏ những dòng dữ liệu có *"Title"* là Nan value

In [5]:
df.dropna(subset = ['Title'], inplace = True)

In [6]:
if df['Title'].isna().sum() == 0:
    print('Column Title is now good !')

Column Title is now good !


##### Có thể thấy số giá bị thiếu ở cột *DiscountPercent* là quá nhiều (hơn 90%) nên ta sẽ tính lại giá trị ở cột *OriginalPrice* sau khi đã áp dụng giảm giá, đổi tên thành *Price* và xóa đi cột *DiscountPercent* 

##### Trước hết, ta sẽ chuyển tất cả giá trị với đơn vị tiền tệ "$" sang "VND", tiến hàng loại bỏ kí tự hiển thị đơn vị tiền tệ, đưa giá trị "Free" thành 0 ở cột *OriginalPrice*

In [7]:
def convert_price(price):
    if price == "Free":
        return 0.0
    elif '₫' in price:
        return float(price[: -4].replace('₫', '').replace('.', '').replace(',', ''))
    elif '$' in price:
        usd_to_vnd = 24265
        return float(price.replace('$', '')) * usd_to_vnd

df['OriginalPrice'] = df['OriginalPrice'].apply(convert_price) 

In [8]:
for price in df['OriginalPrice']:
    print(price)

142000.0
280000.0
100000.0
188000.0
0.0
87500.0
260000.0
150000.0
260000.0
0.0
188000.0
260000.0
45000.0
115000.0
73500.0
260000.0
321000.0
165000.0
120000.0
30000.0
220000.0
165000.0
180000.0
30000.0
363732.35
242407.35
606382.35
242407.35
121082.35
849032.3500000001
242407.35
485057.35
242407.35
606382.35
1455657.35
363732.35
485057.35
363732.35
727707.35
242407.35
387997.35
363732.35
242407.35
242407.35
509322.35
218142.35
727707.35
0.0
606382.35
0.0
48287.35
970357.3500000001
485057.35
121082.35
363732.35
485057.35
121082.35
48287.35
242407.35
606382.35
436527.35
121082.35
363732.35
849032.3500000001
72552.35
485057.35
48287.35
193877.35
242407.35
363732.35
727707.35
242407.35
849032.3500000001
290937.35
165000.0
692000.0
445000.0
140000.0
120000.0
70000.0
130000.0
200500.0
445000.0
165000.0
59000.0
99000.0
260000.0
205000.0
299000.0
205000.0
70000.0
226000.0
142000.0
140000.0
165000.0
142000.0
226000.0
43000.0
115000.0
205000.0
226000.0
205000.0
102000.0
102000.0
102000.0
170000.0

In [9]:
def convert_discount(discount):
    if pd.isna(discount):
        return None
    else:
        return float(discount.replace('-', '').replace('%', '')) * 0.01

df['DiscountPercent'] = df['DiscountPercent'].apply(convert_discount)

In [10]:
for discount in df['DiscountPercent']:
    print(discount)

0.2
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
0.2
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
0.5
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
0.24
nan
nan
nan
nan
nan
nan
0.2
nan
nan
nan
nan
nan
nan
nan
nan
0.3
nan
nan
0.1
nan
nan
nan
nan
nan
nan
0.75
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
0.25
nan
nan
nan
nan
nan
nan
0.25
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
0.5
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
0.2


In [11]:
def price_after_discount(x):
    discount_percent = x['DiscountPercent']
    if pd.isna(discount_percent):
        return x['OriginalPrice']
    else:
        return x['OriginalPrice'] * (1 - discount_percent)

df['Price'] = df.apply(price_after_discount, axis = 1)

In [12]:
df['Price']

0       113600.0
1       280000.0
2       100000.0
3       188000.0
4            0.0
          ...   
2016    337000.0
2017    321000.0
2018    205000.0
2019    200000.0
2020     37500.0
Name: Price, Length: 1734, dtype: float64

In [13]:
df.drop(columns = ['DiscountPercent', 'OriginalPrice'], inplace = True)
df.head(20)

,Title,Genre,Tags,withDLC,isMature,Franchise,ReleaseDate,Developer,Publisher,Languages,PositiveReviews,TotalReviews,NegativeReviews,Price
0,Endless Monday: Dreams and Deadlines,"['Adventure', 'Indie']","['Visual Novel', 'Choose Your Own Adventure', ...",False,False,Endless Monday,"May 4, 2023",hcnone,hcnone,"['English', 'Japanese']",722,722,0.0,113600.0
1,Senren＊Banka,"['Adventure', 'Casual']","['Visual Novel', 'Casual', 'NSFW', 'Dating Sim...",False,True,NaN,"Feb 14, 2020",YUZUSOFT,HIKARI FIELD,"['English', 'Japanese', 'Simplified Chinese', ...","(16,973)","(17,121)",148.0,280000.0
2,A Short Hike,"['Adventure', 'Indie']","['Exploration', 'Adventure', 'Cute', 'Relaxing...",False,False,NaN,"30 Jul, 2019",adamgryu,adamgryu,"['English', 'French', 'Spanish - Latin America...","(14,676)","(14,786)",110.0,100000.0
3,Aseprite,"['Animation & Modeling', 'Design & Illustratio...","['Pixel Graphics', 'Design & Illustration', 'A...",False,False,NaN,"Feb 22, 2016",David Capello,Igara Studio,['English'],"(15,469)","(15,597)",128.0,188000.0
4,Cats Hidden in Paris,"['Casual', 'Indie', 'Free to Play']","['Cats', 'Hidden Object', 'Puzzle', 'Wholesome...",True,False,Travellin Cats,"16 Jul, 2023",Travellin Cats,Travellin Cats,"['English', 'French', 'Italian', 'German', 'Sp...","(11,336)","(11,435)",99.0,0.0
5,Papa's Freezeria Deluxe,"['Action', 'Casual', 'Indie', 'Simulation', 'S...","['Cooking', 'Time Management', 'Simulation', '...",False,False,Papa Louie,"Mar 31, 2023",Flipline Studios,Flipline Studios,['English'],"(6,441)","(6,487)",46.0,87500.0
6,TOEM,"['Adventure', 'Casual', 'Indie']","['Cozy', 'Wholesome', 'Adventure', 'Puzzle', '...",True,False,NaN,"17 Sep, 2021",Something We Made,Something We Made,"['English', 'French', 'German', 'Spanish - Spa...","(4,589)","(4,625)",36.0,260000.0
7,Ib,"['Adventure', 'Indie']","['Horror', '2D', 'RPGMaker', 'Psychological Ho...",False,False,PLAYISM,"Apr 11, 2022",kouri,PLAYISM,"['English', 'Japanese', 'Simplified Chinese', ...","(3,048)","(3,077)",29.0,150000.0
8,Lil Gator Game,"['Action', 'Adventure', 'Casual', 'Indie']","['Casual', 'Cozy', 'Relaxing', 'Cute', 'Wholes...",False,False,Playtonic Friends,"14 Dec, 2022",MegaWobble,Playtonic Friends,"['English', 'French', 'German', 'Portuguese - ...","(3,338)","(3,369)",31.0,260000.0
9,An Arcade Full of Cats,"['Casual', 'Indie', 'Free to Play']","['Cats', 'Hidden Object', 'Relaxing', 'Wholeso...",True,False,Devcats,"16 Nov, 2023",Devcats,Devcats,"['English', 'French', 'Italian', 'German', 'Sp...","(2,711)","(2,736)",25.0,0.0


#### Vì cột Franchise chứa quá nhiều dữ liệu bị thiếu (hơn 50%), nhưng lại không thể "fill" giá trị ngẫu nhiêu nào, hay có giá trị đặc thù nào có thể thay thế những dữ liệu bị thiếu mà không gây nhiễu

Nên ta chọn loại bỏ hẳn cột này ra khỏi dataframe

In [14]:
df.drop(columns = ['Franchise'], inplace = True)

In [15]:
df.head(15)

,Title,Genre,Tags,withDLC,isMature,ReleaseDate,Developer,Publisher,Languages,PositiveReviews,TotalReviews,NegativeReviews,Price
0,Endless Monday: Dreams and Deadlines,"['Adventure', 'Indie']","['Visual Novel', 'Choose Your Own Adventure', ...",False,False,"May 4, 2023",hcnone,hcnone,"['English', 'Japanese']",722,722,0.0,113600.0
1,Senren＊Banka,"['Adventure', 'Casual']","['Visual Novel', 'Casual', 'NSFW', 'Dating Sim...",False,True,"Feb 14, 2020",YUZUSOFT,HIKARI FIELD,"['English', 'Japanese', 'Simplified Chinese', ...","(16,973)","(17,121)",148.0,280000.0
2,A Short Hike,"['Adventure', 'Indie']","['Exploration', 'Adventure', 'Cute', 'Relaxing...",False,False,"30 Jul, 2019",adamgryu,adamgryu,"['English', 'French', 'Spanish - Latin America...","(14,676)","(14,786)",110.0,100000.0
3,Aseprite,"['Animation & Modeling', 'Design & Illustratio...","['Pixel Graphics', 'Design & Illustration', 'A...",False,False,"Feb 22, 2016",David Capello,Igara Studio,['English'],"(15,469)","(15,597)",128.0,188000.0
4,Cats Hidden in Paris,"['Casual', 'Indie', 'Free to Play']","['Cats', 'Hidden Object', 'Puzzle', 'Wholesome...",True,False,"16 Jul, 2023",Travellin Cats,Travellin Cats,"['English', 'French', 'Italian', 'German', 'Sp...","(11,336)","(11,435)",99.0,0.0
5,Papa's Freezeria Deluxe,"['Action', 'Casual', 'Indie', 'Simulation', 'S...","['Cooking', 'Time Management', 'Simulation', '...",False,False,"Mar 31, 2023",Flipline Studios,Flipline Studios,['English'],"(6,441)","(6,487)",46.0,87500.0
6,TOEM,"['Adventure', 'Casual', 'Indie']","['Cozy', 'Wholesome', 'Adventure', 'Puzzle', '...",True,False,"17 Sep, 2021",Something We Made,Something We Made,"['English', 'French', 'German', 'Spanish - Spa...","(4,589)","(4,625)",36.0,260000.0
7,Ib,"['Adventure', 'Indie']","['Horror', '2D', 'RPGMaker', 'Psychological Ho...",False,False,"Apr 11, 2022",kouri,PLAYISM,"['English', 'Japanese', 'Simplified Chinese', ...","(3,048)","(3,077)",29.0,150000.0
8,Lil Gator Game,"['Action', 'Adventure', 'Casual', 'Indie']","['Casual', 'Cozy', 'Relaxing', 'Cute', 'Wholes...",False,False,"14 Dec, 2022",MegaWobble,Playtonic Friends,"['English', 'French', 'German', 'Portuguese - ...","(3,338)","(3,369)",31.0,260000.0
9,An Arcade Full of Cats,"['Casual', 'Indie', 'Free to Play']","['Cats', 'Hidden Object', 'Relaxing', 'Wholeso...",True,False,"16 Nov, 2023",Devcats,Devcats,"['English', 'French', 'Italian', 'German', 'Sp...","(2,711)","(2,736)",25.0,0.0


In [16]:
def format_reviews_col(review):
    regex = r'[(),]'
    review = review.replace('.0', '')
    return re.sub(regex, '', review)

cols_review = ['PositiveReviews', 'TotalReviews', 'NegativeReviews']
df[cols_review] = df[cols_review].applymap(format_reviews_col)
df[cols_review] = df[cols_review].astype(np.int64)

In [17]:
df[cols_review]

,PositiveReviews,TotalReviews,NegativeReviews
0,722,722,0
1,16973,17121,148
2,14676,14786,110
3,15469,15597,128
4,11336,11435,99
...,...,...,...
2016,4834,5282,448
2017,4294,4701,407
2018,4839,5285,446
2019,6486,7384,898


##### Kiểm tra tính toàn vẹn dữ liệu, ta sẽ tính tổng cột positive và negative reviews có bằng giá trị ở cột Total review hay không?

In [18]:
check_review_cols = (df['NegativeReviews'] + df['PositiveReviews'] == df['TotalReviews']).sum()
check_sum = check_review_cols.sum()
if check_sum == df.shape[0]:
    print('Data in Review columns are good !!!')
else:
    print('Have to update the value in Positive or Negative review using Total and another')

Data in Review columns are good !!!


#### Kiểm tra dữ liệu ở cột *Release Date*, nếu giá trị là Nan, đồng nghĩa với trò chơi điện tử này chưa được phát hành chính thức nên xóa khỏi dataframe

In [19]:
df.dropna(subset = 'ReleaseDate', inplace = True)

In [20]:
df[pd.isna(df['Publisher'])]

,Title,Genre,Tags,withDLC,isMature,ReleaseDate,Developer,Publisher,Languages,PositiveReviews,TotalReviews,NegativeReviews,Price
717,Vermillion - VR Painting,"['Casual', 'Indie', 'Simulation']","['VR', 'Sandbox', 'Simulation', 'Design & Illu...",False,False,"26 Jul, 2021",Thomas van den Berge,NaN,"['English', 'French', 'Italian', 'German', 'Sp...",549,572,23,260000.00
1125,Cat Goes Fishing,"['Casual', 'Indie', 'Simulation']","['Fishing', 'Cats', 'Casual', 'Cute', 'Relaxin...",False,False,"19 Jan, 2015",Cat5Games,NaN,['English'],9926,10549,623,90000.00
1163,Severed Steel,"['Action', 'Indie']","['FPS', 'Parkour', 'Bullet Time', 'Action', 'S...",True,True,"17 Sep, 2021",Greylock Studio,NaN,"['English', 'French', 'Italian', 'German', 'Sp...",3490,3670,180,606382.35
1204,Severed Steel,"['Action', 'Indie']","['FPS', 'Parkour', 'Bullet Time', 'Action', 'S...",True,True,"17 Sep, 2021",Greylock Studio,NaN,"['English', 'French', 'Italian', 'German', 'Sp...",3490,3670,180,220000.00
1295,eSail Sailing Simulator,"['Racing', 'Simulation', 'Sports']","['Simulation', 'Sports', 'Racing', 'Sailing']",True,False,"Mar 26, 2019",eSail Ltd.,NaN,"['English', 'French', 'Russian', 'German', 'It...",542,575,33,445000.00
1811,WasteLand Express 废土快递,"['Indie', 'RPG', 'Strategy']","['Strategy', 'Strategy RPG', 'Time Management'...",True,False,"27 Oct, 2022",中子星游戏研究所,NaN,"['English', 'Simplified Chinese']",950,1027,77,58000.00


##### Chuyển cột *ReleaseDate* về kiểu datetime

In [21]:
def convert_to_datetime(date_str):
    formats = [r'%b %d, %Y', r'%d %b, %Y', r'%b %Y']
    for format in formats:
        try:
            return dt.strptime(date_str, format)
        except ValueError:
            pass
    return np.nan

df['ReleaseDate'] = df['ReleaseDate'].apply(convert_to_datetime)

In [22]:
for date in df['ReleaseDate']:
    print(date)

2023-05-04 00:00:00
2020-02-14 00:00:00
2019-07-30 00:00:00
2016-02-22 00:00:00
2023-07-16 00:00:00
2023-03-31 00:00:00
2021-09-17 00:00:00
2022-04-11 00:00:00
2022-12-14 00:00:00
2023-11-16 00:00:00
2021-05-14 00:00:00
2022-03-29 00:00:00
2022-11-09 00:00:00
2022-02-14 00:00:00
2022-10-26 00:00:00
2023-06-10 00:00:00
2022-12-16 00:00:00
2007-08-17 00:00:00
2022-03-23 00:00:00
2022-03-14 00:00:00
2023-10-13 00:00:00
2022-12-20 00:00:00
2018-02-14 00:00:00
2022-01-21 00:00:00
2016-02-26 00:00:00
2011-04-18 00:00:00
2020-09-17 00:00:00
2019-07-23 00:00:00
2022-10-20 00:00:00
2018-10-17 00:00:00
2007-10-10 00:00:00
2021-04-01 00:00:00
2023-10-23 00:00:00
2020-09-03 00:00:00
2020-03-23 00:00:00
2019-04-18 00:00:00
2023-01-26 00:00:00
2020-08-07 00:00:00
2017-10-05 00:00:00
2016-12-01 00:00:00
2021-02-26 00:00:00
2019-03-13 00:00:00
2023-04-01 00:00:00
2017-12-14 00:00:00
2020-06-18 00:00:00
2021-02-11 00:00:00
2017-08-03 00:00:00
2019-10-30 00:00:00
2022-06-16 00:00:00
2020-11-16 00:00:00


In [26]:
type(df['ReleaseDate'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1733 entries, 0 to 2020
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Title            1733 non-null   object        
 1   Genre            1733 non-null   object        
 2   Tags             1733 non-null   object        
 3   withDLC          1733 non-null   object        
 4   isMature         1733 non-null   object        
 5   ReleaseDate      1733 non-null   datetime64[ns]
 6   Developer        1733 non-null   object        
 7   Publisher        1727 non-null   object        
 8   Languages        1733 non-null   object        
 9   PositiveReviews  1733 non-null   int64         
 10  TotalReviews     1733 non-null   int64         
 11  NegativeReviews  1733 non-null   int64         
 12  Price            1733 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int64(3), object(8)
memory usage: 189.5+ KB


##### Ta sẽ sử dụng công thức sau để tính score_rating cho từng trò chơi điện tử dựa trên giá trị ở các cột Reviews với miền giá trị sau khi tính sẽ là [1, 100]
Score = ((Positive Reviews - Negative Reviews) / Total Reviews) * 49.5 + 50.5

Một số lý do để sử dụng công thức này:
1. Tập trung vào các phản hồi tích cực, có nghĩa là mỗi phản hồi tích cực có tác động lớn hơn trên score_rating so với phản hồi tiêu cực
2. Mặc dù công thức tập trung và nhấn mạnh phản hồi tích cực nhưng không bỏ qua bất kì phản hồi tiêu cực nào
3. Có sự cân bằng khi ta cân nhắc phản hổi tích cực thật sự quan trọng hơn nhưng phản hồi tiêu cực cũng có ảnh hưởng nhất định

In [25]:
df['Score'] = ((df['PositiveReviews'] - df['NegativeReviews']) / df['TotalReviews']) * 49.5 + 50.5
# df['Score'] = df['Score'].clip(1, 100)
for i in df['Score']:
    print(i) 

100.0
99.14420886630454
99.26349249289868
99.1875360646278
99.14289462177524
99.2979805765377
99.2294054054054
99.06694832629185
99.08904719501336
99.09539473684211
99.12818003913894
99.3265306122449
99.51785714285714
99.32036613272311
99.39310344827587
99.43589743589743
99.84372533543805
98.9851936218679
99.45901639344262
99.18374558303887
99.26485148514851
99.38699690402477
99.78686759956943
99.55203619909503
98.26849113249784
98.7534764576726
98.4943486369256
98.73005135630416
98.63172839676348
98.12183859132574
98.52043262568594
97.8950103135852
98.71780963133867
98.12522191975381
98.39546951638509
98.1369576581286
98.65670250450563
98.65350364963504
97.97950828146648
98.27548387096775
98.46732863549008
97.97562141491395
98.22850445918134
97.92445106066245
98.81999857255013
98.65914221218961
98.09070910024198
98.66472400299725
98.35050675675676
98.96623401971308
98.37880151160698
98.50328909405656
98.28534578061664
98.28788235294118
98.0639191494947
98.77740963855422
98.36712379540